In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.optimize import curve_fit
from scipy.integrate import cumulative_trapezoid
from scipy.interpolate import CubicSpline

In [ ]:
class IsingModel:
    def __init__(self, L, T):
        #parametros do modelo
        self.L = L
        self.T = T

        #array com os spins
        self.spins = np.ones((L,L))

        #arrays para guardar evoluções das variáveis
        self.energies= []
        self.magnetizations= [-L**2]

        #energia e magnetização total
        self.energy_total = self.calc_ener_total()
        self.magnet_total = np.sum(self.spins)

    def calc_ener_spin(self, i, j):
        #calcular a energia de um spin
        spin= self.spins[i,j]
        neighbours= (
        self.spins[(i+1)%self.L, j] + 
        self.spins[i, (j+1)%self.L] + 
        self.spins[(i-1)%self.L, j] + 
        self.spins[i, (j-1)%self.L]
        )
        energy=-spin*neighbours
        return energy
    
    def calc_ener_total( self ):
        energy=0
        for i in range(self.L):
            for j in range(self.L):
                energy+= -self.spins[i,j]*(self.spins[(i+1)%self.L,j]+ self.spins[i,(j+1)%self.L])
        return energy
    
    def calc_ener ( self ):
        energy=self.energy_total/self.L**2

        return energy
    
    def calc_mag( self ):
        mag = self.magnet_total
        return mag
    
    def iter_monte_carlo ( self, n_iter ):

        for i in tqdm (range(n_iter), desc=f"L={self.L:6d}, T={self.T:8f}"):
    
    @property
    def energy(self):
        return np.array(self.energies)
    


IndentationError: expected an indented block after 'for' statement on line 48 (1192968361.py, line 50)

## 1. Tempo de Termalização

## 2. Determinação da Temperatura Crítica

## 3. Determinação da Energia Livre de Helmholtz, F